In [1]:

import os

import cv2
import ipywidgets as widgets
import torch
import torchvision
from IPython.display import display
import PIL


In [2]:
model_road = torchvision.models.resnet18(pretrained=False)
model_road.fc = torch.nn.Linear(512, 2)
model_road.load_state_dict(torch.load('best_steering_model_xy.pth', map_location=torch.device('cpu')))

/home/talsema/.conda/envs/thu/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/talsema/.conda/envs/thu/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [3]:
def get_road_direction(image) -> (float, float):
    model_xy = model_road(image).detach().float().cpu().numpy().flatten()
    model_x = model_xy[0]
    model_y = (0.5 - model_xy[1]) / 2.0
    return model_x, model_y

In [4]:
img_list = []
for root, dirs, files in os.walk("img", topdown=False):
    for name in files:
        img_list.append(root + "/" + name)

In [10]:
counter = 0

btn = widgets.Button(
    description="Next"
)

display(btn)


def on_click(b):
    global counter
    image = PIL.Image.fromarray(img_list[counter])
    display(image)
    counter += 1

btn.on_click(on_click)


Button(description='Next', style=ButtonStyle())